In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import acnportal
import sklearn
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns

from copy import deepcopy
import warnings
import pytz
import time
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timedelta
from enum import Enum
from collections import namedtuple, defaultdict
import gzip
import random
import os
import json

from acnportal import acnsim
from acnportal import algorithms
from acnportal.acnsim.events import EventQueue
# from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from utility_functions.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge
from modified_simulator import modified_simulator
# from acnportal.acnsim import analysis
from utility_functions import analysis
# from modified_evse import *
from acnportal.acnsim.models.evse import get_evse_by_type
# import sqlite3
# from sqlalchemy import create_engine

In [3]:
# This method won't take a evse_per_phase argument
def ev_fleet_level_2_network(transformer_cap=30):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'
    # evse_type = 'ClipperCreek'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(3)]
    BC_ids = ['BC-{0}'.format(i) for i in range(3)]
    CA_ids = ['CA-{0}'.format(i) for i in range(3)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [4]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts      

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

#### Generate synthetic events for simulation

In [5]:
# Generate synthetic events based on JPL data provided by ACN-Sim.
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    # gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=5)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [6]:
# 27 EV sessions per weekday
events_27_ev = get_synth_events(27)

c:\Users\s3955218\Anaconda3\envs\evsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# Checking session details
session_copy = deepcopy(events_27_ev)
# print(type(session_copy))
session_list = list(session_copy.queue)
for session in session_list:
    print(session[0], session[1].ev._session_id, session[1].ev.arrival, \
          session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type)

641 session_0 641 701 7.934345938901481 Plugin
649 session_10 649 713 10.323793893738596 Plugin
645 session_13 645 776 7.12019424079705 Plugin
656 session_16 656 781 6.435302930028509 Plugin
650 session_7 650 793 11.889061750634093 Plugin
654 session_24 654 764 7.109563307599831 Plugin
660 session_2 660 770 14.304548254456625 Plugin
657 session_15 657 768 9.457474389868311 Plugin
662 session_17 662 777 4.710936250418977 Plugin
721 session_4 721 781 2.051809006318214 Plugin
670 session_9 670 730 9.455162929537778 Plugin
666 session_5 666 795 12.028619121166841 Plugin
677 session_12 677 798 5.544695375821384 Plugin
670 session_6 670 773 12.334714552944684 Plugin
679 session_14 679 780 6.004555444377474 Plugin
727 session_1 727 787 13.296160475677045 Plugin
665 session_8 665 725 10.655258720160866 Plugin
670 session_3 670 757 10.143353213457498 Plugin
682 session_18 682 784 5.087004243366981 Plugin
729 session_19 729 789 6.4020639999844535 Plugin
738 session_20 738 816 10.391241543441506 

#### Algorithms and new equation

In [15]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['RR'] = algorithms.RoundRobin(algorithms.first_come_first_served, continuous_inc=1)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_without_priority_ev),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_for_priority_ev),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=300, max_recompute=1)

#### Run Simulation with New Equation

In [16]:
sims = dict()
def run_experiment(alg_name, cap):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    # Australian timezone for the experiment
    timezone = pytz.timezone('Australia/Melbourne')

    start = timezone.localize(datetime(2025, 4, 8))
    end = timezone.localize(datetime(2025, 4, 8))

    cn = ev_fleet_level_2_network(transformer_cap=cap)

    alg = deepcopy(sch[alg_name])
    alg.max_rate_estimator = algorithms.SimpleRampdown()
    alg.estimate_max_rate = True
    experiment_events = deepcopy(events_27_ev)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = modified_simulator.Simulator(cn, alg, experiment_events, start, period=PERIOD, signals=signals, verbose=False)
    print("Running...")
    start_simulation = time.time()
    if alg_name == "MPC_Offline":
        alg.register_events(experiment_events)
        alg.solve()
    # if alg_name == "MPC":
    #     sim.high_priority_ev_sessions = ["session_19", "session_7", "session_15", "session_10"]
    # sim.high_priority_ev_sessions = ["session_4", "session_12", "session_2"]
    sim.run()
    print(f"Run time: {time.time() - start_simulation}")

    return sim

In [17]:
warnings.simplefilter("ignore")

capacities = list(range(80, 110, 5))
alg_names = ["Unctrl", "RR", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            try:
                sims[config] = run_experiment(*config)
            except Exception as e:
                print(f"InfeasibilityException: {e}")
                sims[config] = None
                continue

('Unctrl', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 0.36624813079833984
('RR', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 1.5952272415161133
('MPC', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 18.270440578460693
('Unctrl', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 0.3675410747528076
('RR', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 1.5383667945861816
('MPC', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 13.05818796157837
('Unctrl', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 0.34013843536376953
('RR', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 1.4599761962890625
('MPC', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 12.762465476989746
('Unctrl', 95)
76.30522088353413
137.68115942028987
Running...
Run time: 0.2826261520385742
('RR', 95)
76.30522088353413
137.68115942028987
Running...
Run time: 1.3659636974334717
('MPC', 95)
76.

In [6]:
# result_dir = "results/sims/new_simulation"
# if not os.path.exists(result_dir):
#     os.makedirs(result_dir)
    
# for config, sim in sims.items():
#     name = "results/sims/new_simulation/{0}-{1}.json.gz".format(*config)
#     if not os.path.exists(name):
#         data = sim.to_json()
#         with gzip.GzipFile(name, 'w') as fout:
#             fout.write(json.dumps(data).encode('utf-8'))

#### Result Analysis

In [18]:
def calc_metrics(config, sim):
    metrics = {
        "Network Type": "three_phase",
        "Algorithm": config[0],
        "Capacity (kW)": config[1],
        "Energy Delivered (%)": analysis.proportion_of_energy_delivered(sim) * 100,
        "Demand met": analysis.proportion_of_demands_met(sim) * 100,
        "Max Utilization (%)": np.max(analysis.aggregate_power(sim)) / config[1] * 100,
        "energy_cost": analysis.energy_cost(sim),
        "total_energy_delivered": analysis.total_energy_delivered(sim),
        "Energy Delivered to priority evs (%)": analysis.proportion_of_priority_evs_energy_delivered(sim, ["session_7",
                                                                                                           "session_2",
                                                                                                           "session_9",
                                                                                                           "session_3",
                                                                                                           "session_11",
                                                                                                           "session_8",
                                                                                                           "session_21",
                                                                                                           ]),
        "Peak (kW)": np.max(analysis.aggregate_power(sim))
    }
    metrics["Current Unbalance"] = np.nanmean(analysis.current_unbalance(sim, ['Secondary {0}'.format(p) for p in 'ABC'], 'NEMA'))

    return metrics

In [19]:
# filter the sims dict to only include the ones that are not None
sims = {k: v for k, v in sims.items() if v is not None}

In [20]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [21]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Energy Delivered to priority evs (%),Peak (kW),Current Unbalance
0,three_phase,Unctrl,80,100.000,100.000,82.500,16.833,232.527,100.000,66.000,0.417
1,three_phase,RR,80,93.102,92.593,71.863,16.107,216.487,85.358,57.490,0.400
2,three_phase,MPC,80,83.539,55.556,61.481,15.117,194.252,85.551,49.185,0.444
3,three_phase,Unctrl,85,100.000,100.000,77.647,16.844,232.527,100.000,66.000,0.428
4,three_phase,RR,85,93.102,92.593,67.635,16.107,216.487,85.358,57.490,0.433
5,three_phase,MPC,85,83.621,55.556,61.041,15.134,194.442,85.551,51.885,0.448
6,three_phase,Unctrl,90,100.000,100.000,73.333,16.833,232.527,100.000,66.000,0.435
7,three_phase,RR,90,93.102,92.593,63.878,16.107,216.487,85.358,57.490,0.438
8,three_phase,MPC,90,83.621,55.556,57.650,15.134,194.442,85.551,51.885,0.432
9,three_phase,Unctrl,95,100.000,100.000,69.474,16.844,232.527,100.000,66.000,0.448


#### Simulation with different algorithms

In [22]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['LLF'] = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_for_priority_ev, 2),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_without_priority_ev),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

In [23]:
sims = dict()

In [24]:
warnings.simplefilter("ignore")

capacities = list(range(80, 110, 5))
alg_names = ["Unctrl", "LLF", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 0.29233360290527344
('LLF', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 0.7428395748138428
('MPC', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 12.574277639389038
('Unctrl', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 0.3168494701385498
('LLF', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 0.7001943588256836
('MPC', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 12.58226203918457
('Unctrl', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 0.32032179832458496
('LLF', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 0.7000992298126221
('MPC', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 12.819813013076782
('Unctrl', 95)
76.30522088353413
137.68115942028987
Running...
Run time: 0.30994677543640137
('LLF', 95)
76.30522088353413
137.68115942028987
Running...
Run time: 0.7914121150970459
('MPC', 95

In [26]:
# filter the sims dict to only include the ones that are not None
sims = {k: v for k, v in sims.items() if v is not None}

In [27]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [28]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Energy Delivered to priority evs (%),Peak (kW),Current Unbalance
0,three_phase,Unctrl,80,100.000,100.000,82.500,16.844,232.527,100.000,66.000,0.438
1,three_phase,LLF,80,93.102,92.593,71.863,16.107,216.487,85.358,57.490,0.421
2,three_phase,MPC,80,83.636,55.556,64.856,15.137,194.477,85.551,51.885,0.426
3,three_phase,Unctrl,85,100.000,100.000,77.647,16.844,232.527,100.000,66.000,0.448
4,three_phase,LLF,85,93.102,92.593,67.635,16.107,216.487,85.358,57.490,0.418
5,three_phase,MPC,85,83.621,55.556,61.041,15.134,194.442,85.551,51.885,0.423
6,three_phase,Unctrl,90,100.000,100.000,73.333,16.844,232.527,100.000,66.000,0.429
7,three_phase,LLF,90,93.102,92.593,63.878,16.107,216.487,85.358,57.490,0.417
8,three_phase,MPC,90,83.621,55.556,57.650,15.134,194.442,85.551,51.885,0.421
9,three_phase,Unctrl,95,100.000,100.000,69.474,16.833,232.527,100.000,66.000,0.405
